In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import pathlib
from tensorflow import keras
import os
from keras.callbacks import ModelCheckpoint
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn import datasets
from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from keras.layers import Input, Dense    #using set model component
from keras.models import Model    #using set model 
from keras.utils import plot_model    #show model structure
from keras import layers as Layer
import keras 
from pandas import DataFrame as df
from keras.callbacks import EarlyStopping
import gzip
import pickle
from keras import backend as K
import tensorflow as tf
from keras.layers import Dense, Dropout, Input, Activation, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
import csv
from scipy import stats

/home/srkim/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/srkim/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/srkim/anaconda3/envs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/srkim/anaconda3/envs/keras/lib/python3.6/site-

In [2]:
def check_correct(predict, y):
    result = {}
    result['True-Positive'] = 0
    result['True-Negative'] = 0
    result['False-Negative'] = 0
    result['False-Positive'] = 0

    for i in range(len(predict)) :
        if predict[i] == y[i] :
            if y[i] == 0 :
                result['True-Negative'] += 1
            else :
                result['True-Positive'] += 1
        else :
            if y[i] == 0 :
                result['False-Positive'] += 1
            else :
                result['False-Negative'] += 1

    for result_k, result_v in result.items():
        print(result_k +" : "+ str(result_v))
    
    acc=(result['True-Positive']+result['True-Negative'])/len(y)
    sensitivity=result['True-Positive']/(result['True-Positive']+result['False-Negative'])
    specificity=result['True-Negative']/(result['True-Negative']+result['False-Positive'])
    print("Accuracy :", acc)
    print("Sensitivity :", sensitivity)
    print("Specificity :", specificity)
    return acc, sensitivity, specificity

In [12]:
def model_performance(x, model, y):
    hypo = model.predict_proba(x)
    pred = np.where(df(hypo)[1] > 0.5, 1, 0).flatten()
    acc, sen, spe=check_correct(pred, y)
    auc=metrics.roc_auc_score(y, df(hypo)[1])
    
    df_hypo=df(hypo)[1]
    df_hypo.columns=['hypothesis 1']
    
    df_pred=df(pred)
    df_pred.columns=['prediction']
    
    df_y=df(y)
    df_y.columns=['y']
    
    pred_result=pd.concat([df_y,df_hypo, df_pred],axis=1)
    
    print("Accuracy : ",acc)
    print("AUC : ",auc)
    print(" ")
    print(df(pd.crosstab(y, pred, rownames=['Actual'], colnames=['Predicted'], margins=True)))
    print(" ")
    # print(df(pred_result))

    return acc ,sen, spe ,auc, pred_result

In [4]:
def feature_importance_last(model, column_names) :
    #calcuate feature importances
    importance = model.feature_importances_
    #Feature extraction
    df_name=pd.DataFrame(column_names)
    df_name.columns=['name']
    f_imp=pd.DataFrame(importance.tolist())
    f_imp.columns=['importance']
    importance_data = pd.concat([df_name,f_imp],axis=1).sort_values(['importance'],ascending=0)
    
    return importance_data.tail(1)


In [5]:
def model_performance_DNN(x, model, y):
    hypo = model.predict_classes(x)
    pred = np.where(hypo > 0.5, 1, 0).flatten()
    acc, sen, spe=check_correct(hypo, y)
    auc=metrics.roc_auc_score(y, hypo)
    
    df_hypo=df(hypo)
    df_hypo.columns=['hypothesis 1']
    
    df_pred=df(pred)
    df_pred.columns=['prediction']
    
    df_y=df(y)
    df_y.columns=['y']
    
    pred_result=pd.concat([df_y,df_hypo, df_pred],axis=1)
    
    print("Accuracy : ",acc)
    print("AUC : ",auc)
    print(" ")
    print(df(pd.crosstab(y, pred, rownames=['Actual'], colnames=['Predicted'], margins=True)))
    print(" ")
    print(df(pred_result))

    return acc ,sen, spe ,auc, pred_result

### Ensemble Model
##### Classification 1

In [6]:
dataframe = pd.read_csv("/nfs-data/IRIS/data/IRIS_train.csv")
dataframe

,SampleID,SSPG,SSPG_classification,A1C,GLU,AG,ALB.x,ALKP,BASO,BASOAB,...,ZSWIM7,ZSWIM8,ZSWIM8.AS1,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3,index
0,ZJTKAE3-02,162.0,1,5.5,89,6,3.9,99,0.5,0.03,...,3.885177,5.147299,7.673803,0.000000,0.000000,0.000000,2.151853,0.000000,0.755996,1
1,ZJTKAE3-04,162.0,1,5.2,98,8,4.1,80,0.7,0.03,...,4.259629,8.062151,8.216878,2.009271,0.000000,2.009271,2.009271,0.000000,0.000000,2
2,ZJTKAE3-06,162.0,1,5.5,91,12,4.0,67,0.9,0.05,...,4.434087,5.784487,7.768989,0.000000,1.552295,0.717454,2.462038,0.717454,0.717454,3
3,ZJTKAE3-2012,162.0,1,5.1,94,5,4.2,83,1.1,0.05,...,2.076022,5.602978,7.817027,1.193186,0.000000,0.716583,2.076022,1.193186,0.716583,2
4,ZJTKAE3-2013,162.0,1,5.0,92,9,4.2,73,0.6,0.04,...,2.538052,5.708347,8.232187,1.767240,1.379998,0.000000,3.513165,0.849071,0.849071,2
5,ZJTKAE3-2015,162.0,1,5.0,92,6,4.1,80,1.0,0.05,...,2.562367,6.576751,7.487707,0.690055,0.690055,0.000000,2.704896,1.154901,0.690055,2
6,ZJTKAE3-6021,162.0,1,5.1,82,10,3.9,68,1.2,0.06,...,4.069730,8.760586,8.187742,2.056051,0.000000,0.000000,2.056051,0.000000,0.000000,4
7,ZJXC41N-03,75.0,0,5.3,96,5,4.0,103,0.9,0.02,...,0.860751,6.279888,8.179203,0.860751,0.860751,0.000000,2.092185,0.000000,0.000000,3
8,ZK112BX-01,234.0,1,5.9,112,5,4.2,95,0.5,0.02,...,2.687263,7.452109,8.016244,0.000000,0.000000,1.062246,2.420273,1.062246,2.687263,2
9,ZK112BX-07,234.0,1,6.0,109,4,4.4,76,0.7,0.03,...,2.771067,6.261900,8.184130,0.000000,1.296623,0.000000,3.050093,2.424813,0.000000,4


In [7]:
# rfc = RandomForestClassifier(n_estimators=350, max_depth=5, min_samples_split=4, random_state=10)
# abc = AdaBoostClassifier(n_estimators=250, random_state=0, learning_rate=0.05)
gbc = GradientBoostingClassifier(n_estimators=350, learning_rate=0.05, random_state=10, min_samples_split=5)
# xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=5,learning_rate=0.05, random_state=10)

In [8]:
value= dataframe.drop(["SampleID", "SSPG_classification", "SSPG", "index"],axis=1)
interest = dataframe.SSPG_classification

print("Information of Training Data")
print("Samples : {}".format(value.shape[0]))
print("Features : {}".format(value.shape[1]))

Information of Training Data
Samples : 343
Features : 10783


In [9]:
#MinMax
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

X = np.array(value)
min_max_scaler = MinMaxScaler()
X_MinMax = min_max_scaler.fit_transform(X)

In [10]:
train_x, test_x, train_y, test_y = train_test_split(X_MinMax, interest, test_size=0.2, random_state=10, stratify=interest)

In [11]:
gbc.fit(train_x, train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=5,
                           min_weight_fraction_leaf=0.0, n_estimators=350,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=10, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [13]:
#calcuate feature importances
importance = gbc.feature_importances_
#Feature extraction
df_name=pd.DataFrame(value.columns)
df_name.columns=['name']
f_imp=pd.DataFrame(importance.tolist())
f_imp.columns=['importance']
importance_data2 = pd.concat([df_name,f_imp],axis=1).sort_values(['importance'],ascending=0)
#Export to csv
importance_data2.to_csv("/nfs-data/IRIS/data/feature_importance_0604.csv")

In [14]:
imp_feature = importance_data2[importance_data2["importance"]!=0].drop(["importance"],axis=1).name

In [15]:
#t_p_value = 0.001

# 원하는 모델 선택
data_type="GradientBoost" #"RF", "AdaBoost", "GradientBoost", "XGBoost"
model = gbc

In [16]:
for test_index in range(1,6):
    data = dataframe

    # 5-fold cross vaildation
    train_data = data[data['index'] != test_index]
    test_data = data[data['index'] == test_index]
    
#     # t.test 
#     train_0 = train_data[train_data["SSPG_classification"]==0].drop(["SampleID", "SSPG_classification", "SSPG", "index"],axis=1)
#     train_1 = train_data[train_data["SSPG_classification"]==1].drop(["SampleID", "SSPG_classification", "SSPG", "index"],axis=1)
    
#     micro_list = train_0.columns
    
#     sub_micro=[]
#     for m in micro_list:
#         t_result=stats.ttest_ind(train_0[m],train_1[m])
#         if t_result[1]<t_p_value:
#             sub_micro.append(m)
                
#     train_x = train_data[sub_micro]
#     train_y = train_data.SSPG_classification.values.reshape(-1,1).ravel()
#     test_x = test_data[sub_micro]
#     test_y = test_data.SSPG_classification.values.reshape(-1,1).ravel()

    train_x = train_data[imp_feature]
    train_y = train_data.SSPG_classification.values.reshape(-1,1).ravel()
    test_x = test_data[imp_feature]
    test_y = test_data.SSPG_classification.values.reshape(-1,1).ravel()


    
    # MinMax scale
    x_scaler=MinMaxScaler()
    x_scaler.fit(train_x)
    nor_train_x=x_scaler.transform(train_x)
    
    df_train_x=pd.DataFrame(nor_train_x,columns=train_x.columns)
    
    tx_scaler=MinMaxScaler()
    tx_scaler.fit(test_x)
    nor_test_x=tx_scaler.transform(test_x)
    
    # save data
    if test_index == 1:
        my_col  = ["test_index","feature_num","except_feature","train_acc","train_sen","train_spe","train_AUC","test_acc","test_sen","test_spe","test_AUC"]

        # Need to change the name each combination
        output_file = "./result/"+data_type+"_feature_reduction_0604_feature_466.csv"
        csv_out_file = open(output_file, 'w', newline = '')
        filewriter = csv.writer(csv_out_file)
        filewriter.writerow(my_col)
        csv_out_file.close()
    

    # 변수 초기화
    sub_train_x = nor_train_x
    sub_test_x = nor_test_x
    column_names = df_train_x.columns

    for col_num in range(0,len(df(df_train_x).columns)):
        feature_num = len(df(df_train_x).columns) - col_num

        model.fit(sub_train_x, train_y)
        train_acc, train_sen, train_spe, train_auc, _ = model_performance(sub_train_x,model,train_y)
        test_acc, test_sen, test_spe, test_auc, _ = model_performance(sub_test_x,model,test_y)

        temp_output = [test_index,feature_num,feature_importance_last(model, column_names).iloc[0,0], train_acc, train_sen, train_spe, train_auc, test_acc, test_sen, test_spe, test_auc]
        print(temp_output)
        csv_out_file = open(output_file, 'a', newline = '')
        filewriter = csv.writer(csv_out_file)
        filewriter.writerow(temp_output)
        csv_out_file.close()


        column_names = pd.DataFrame(column_names).drop(feature_importance_last(model, column_names).index, axis='rows')
        column_names.reset_index(drop=True, inplace=True)
        sub_train_x = pd.DataFrame(sub_train_x).drop(feature_importance_last(model, column_names).index, axis='columns')
        sub_train_x.columns = column_names.index
        sub_test_x = pd.DataFrame(sub_test_x).drop(feature_importance_last(model, column_names).index, axis='columns')
        sub_test_x.columns = column_names.index


True-Positive : 131
True-Negative : 143
False-Negative : 0
False-Positive : 0
Accuracy : 1.0
Sensitivity : 1.0
Specificity : 1.0
Accuracy :  1.0
AUC :  1.0
 
Predicted    0    1  All
Actual                  
0          143    0  143
1            0  131  131
All        143  131  274
 
True-Positive : 31
True-Negative : 36
False-Negative : 2
False-Positive : 0
Accuracy : 0.9710144927536232
Sensitivity : 0.9393939393939394
Specificity : 1.0
Accuracy :  0.9710144927536232
AUC :  0.994949494949495
 
Predicted   0   1  All
Actual                
0          36   0   36
1           2  31   33
All        38  31   69
 
[1, 466, 'NASP', 1.0, 1.0, 1.0, 1.0, 0.9710144927536232, 0.9393939393939394, 1.0, 0.994949494949495]
True-Positive : 131
True-Negative : 143
False-Negative : 0
False-Positive : 0
Accuracy : 1.0
Sensitivity : 1.0
Specificity : 1.0
Accuracy :  1.0
AUC :  1.0
 
Predicted    0    1  All
Actual                  
0          143    0  143
1            0  131  131
All        143  131  274

KeyboardInterrupt: 